In [1]:
import pandas as pd
import numpy as np
import os
import csv

In [2]:
#Event='AscendEx'
Event='Normal'
raw_path='../Dataset/Normal/'
account_feature_output_path = '../Code_Results/Ac_Balance_Results/'+Event+'_account_Tx_Internal.csv'
feature_output_path ='../Code_Results/Intermediary_Account_Feature_Results/'+Event

In [3]:
file_list = os.listdir(raw_path)
normal_list=[]
for k in range(0,len(file_list)):
    normal_list.append(file_list[k][:-4])

In [4]:
with open(account_feature_output_path,'w') as f:

    csv_write = csv.writer(f)
    csv_head = ["address","Tx_Internal"]
    csv_write.writerow(csv_head)

output = pd.read_csv(account_feature_output_path)
output['address'] = normal_list


In [5]:
#Step3：统计每个账户的交易速度平均值
import math
for i in range(0,len(normal_list)):

    Del_T=[]
    print('%d/%d'%(i+1,len(normal_list)),end="\r")
    file_name=normal_list[i]
    Ac=file_name
    file_name = raw_path + file_name + '.csv'
    #print(file_name,end="\r")
    if os.path.exists(file_name):
        record = pd.read_csv(file_name)
        Ac_Tx=record.loc[record["isSuccessful"]=='OK']
    
    for j in range(0,len(Ac_Tx)-1):
        Tx=Ac_Tx.iloc[j]
        if Tx['fromac']==Ac:
            Next_Tx=Ac_Tx.iloc[j+1]
            if Next_Tx['toac']==Ac:
                if Next_Tx['timestamp']-Tx['timestamp']<604800:
                    Del_T.append(Next_Tx['timestamp']-Tx['timestamp'])
        if Tx['toac']==Ac:
            Next_Tx=Ac_Tx.iloc[j+1]
            if Next_Tx['fromac']==Ac:
                if Next_Tx['timestamp']-Tx['timestamp']<604800:
                    Del_T.append(Next_Tx['timestamp']-Tx['timestamp'])
    if len(Del_T) !=0:
        Avg_Internal=sum(Del_T)/len(Del_T)
        output['Tx_Internal'][i]=Avg_Internal  
    if len(Del_T) ==0:
        output['Tx_Internal'][i]=0

In [8]:
output

,address,Tx_Internal
0,0x0000000000000000000000000000000000000000,0
1,0x0000000d2138b2ee1865cd3e4454c562b9ba7bff,20.737
2,0x0000002bfa4dac0fdcf5cd9d47259241ce92459b,8.62725
3,0x000000809f5115368b36a7aaa4480caaf5bf809c,54.362
4,0x000001f568875f378bf6d170b790967fe429c81a,32.718
...,...,...
11309,0x1235c4a48841724130abccfbd13e64e1f150bc5b,123.859
11310,0x12362e9d5a0f816b6d4a755b1b14158028309d03,0.0822222
11311,0x123641357e08723be9db36967184412d5d7b07dc,0.706406
11312,0x123658dadbf5d26a33736485b7564a871c05d473,0.0563889


In [7]:
output['Tx_Internal']=output['Tx_Internal']/(60*60)#交易的间隔为小时

In [9]:
output=output.loc[output['Tx_Internal']!=0]
output.to_csv('../Code_Results/Tx_Velocity_Results/'+Event+'_Tx_Internal.csv', index=False)

In [34]:
output['Tx_Internal'].mean()

20.17555956775148

In [25]:
max(output['Tx_Internal'])

167.97

In [26]:
min(output['Tx_Internal'])

0.00018518518518518518

In [28]:
np.median(output['Tx_Internal'])

14.4365

In [32]:
output['Tx_Internal'].mode()[0]

0.0125

In [33]:
output['Tx_Internal'].median()

14.4365